In [4]:
import pandas as pd 

In [5]:
#importation du dataset
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")

C:\Users\USER\AppData\Local\Temp\ipykernel_11536\1282787046.py:2: DtypeWarning: Columns (36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


In [6]:
#notre objectif va être de se baser sur ce dataset et de créer un modèle.
#On va faire un module par partie nécessaire de notre modèle. et on save indépendamment du github de notre côté afin
#d'être sur à la fin de notre travail aujourd'hui. On télécharge data_exploration à l'aide de file puis Save AS..
#on va coder tout notre modèle ici afin de simplifier pour l'instant. 

In [7]:
#a. Récupération des datasets
#on a récupérer au dessus notre dataset
#test de bonne récupération. Et que l'on pourra facilement le gérer par la suite:
#un brin de feature engineering pour cela:
dataset.info()
#On a bien notre dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   as_of_year                      28632 non-null  int64  
 1   respondent_id                   28632 non-null  object 
 2   agency_name                     28632 non-null  object 
 3   agency_abbr                     28632 non-null  object 
 4   agency_code                     28632 non-null  int64  
 5   loan_type_name                  28632 non-null  object 
 6   loan_type                       28632 non-null  int64  
 7   property_type_name              28632 non-null  object 
 8   property_type                   28632 non-null  int64  
 9   loan_purpose_name               28632 non-null  object 
 10  loan_purpose                    28632 non-null  int64  
 11  owner_occupancy_name            28632 non-null  object 
 12  owner_occupancy                 

In [8]:
#b. Importations 
#Que l'on remplira au fur et à mesure.
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score, validation_curve, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [ ]:
#c. Feature Engineering
#Rien d'implémenter de connu pour faire ce que l'on doit faire. 
#Le gros du travail qui va être fait: 
#On va partir du principe faut que l'on s'assure:
#les colonnes dont nous devons nous occuper:

#Eléments à supprimer: 
#On va enlever les éléments qui ont des cases  vides dans: 
#C'est que ce sont des logements saisonniers/temporaires dans une zone de l'alaska sous sous peuplé.
#Que 75000     habitants sur plusieurs millions. 
#Sinon on perd des données c'est dommage et si on les remplace par des 0 cela risque de mal influencer notre modèle. 
#ceux qui n'ont pas de county_code 
#et pour le:
#applicant_income_000s
#On doit supprimer les éléments qui ont vide pour cette valeur. 
#Peut être le mettre plus haut pour simplifier les boucles for précédentes. 
n=len(dataset)
print(n)


#les colonnes à supprimer: on va en récupérer les noms 
#on va récupérer les inutiles et celles qui forment des doublons. Par exemple on a des colonnes dont on a du data 
#categorical et ensuite sa transcription en numerical. On garde alors juste la transcription en numerical
#Les colonnes que l'on enlève et l'explication qui va avec: 
# Les inutiles
#la première: as_of_year, respondent_id,  state_abbr, state_code, hoepa_status, edit_status, sequence_number,
#application_date_indicator
dataset= dataset.drop(columns=["as_of_year", "respondent_id",  "state_abbr", "state_code", "hoepa_status", "edit_status", "sequence_number"])

#Toutes celles où l'on a la transposition en numerical ensuite:
#agency_name, agency_abbr, loan_type_name, property_ype_name, loan_purpose_name, owner_occupancy_name,
#preapproval_name, action_taken_name, msamd_name, county_name, applicant_ethnicity_name, co_applicant_ethnicity_name,
#applicant_race_name_1, applicant_race_name_2, applicant_race_name_3, applicant_race_name_4, applicant_race_name_5
#co_applicant_race_name_1, co_applicant_race_name_2, co_applicant_race_name_3, co_applicant_race_name_4,
#co_applicant_race_name_5, applicant_sex_name, co_applicant_sex_name, purchaser_type_name, 
#denial_reason_name_1, denial_reason_name_2, denial_reason_name_3, hoepa_status_name, lien_status_name
#edit_status_name, state_name,
#toutes les colonnes où il y a name en fait. 
dataset= dataset.drop(columns=["agency_name", "agency_abbr", "loan_type_name", "property_type_name", "loan_purpose_name", "owner_occupancy_name",
    "preapproval_name", "action_taken_name", "msamd_name", "county_name", "applicant_ethnicity_name", "co_applicant_ethnicity_name",
    "applicant_race_name_1", "applicant_race_name_2", "applicant_race_name_3", "applicant_race_name_4", "applicant_race_name_5",
    "co_applicant_race_name_1", "co_applicant_race_name_2", "co_applicant_race_name_3", "co_applicant_race_name_4",
    "co_applicant_race_name_5", "applicant_sex_name", "co_applicant_sex_name", "purchaser_type_name", 
    "denial_reason_name_1", "denial_reason_name_2", "denial_reason_name_3", "hoepa_status_name", "lien_status_name",
    "edit_status_name", "state_name"])


#que toutes les colonnes soient remplies. Quitte à en supprimer certaines ou en créer de nouvelles.

#A mettre des 1 dans les vides cases vides pour:    
# msamd, county_code, census_tract_number
#Pourquoi : 
#pour msamd les colonnes non remplis c'est que le logement ne se trouve pas dans une métropole de l'Alaska
#On peut alors convenir d'un nombre pour ce fait de ne pas être dans une métropole. 
#pour les county de même on a tout une partie de l'alaska qui n'appartient à aucun coutny et est dirigé directement
#par l'état. 
#Attention toujours deux valeurs vides dans notre tableau sur excel. A faire gaffe si pareil sur csv. 
#On va créer la fonction qui va modifier nos colonnes. Valeur par valeur
def f(a):
    if a==None: 
            #on met 1 à place
            a=1
#et ensuite modifier chaque colonne avec 
dataset[["msamd", "county_code", "census_tract_number"]] = dataset[["msamd", "county_code", "census_tract_number"]].applymap(f)

#pour la race de l'applicant. 
# On va utiliser oneHotencoder
# On ne va pas faire comme dans le dataset. On va créer 6 nouvelles features.
#Car on a quand même plus de 500 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
#Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 

"""#Et sinon bien sur on met un 1 dans notre provided. 
n=len(dataset)
print(n)
#On va créer nos 6 nouvelles colonnes:
#de longueur n et de noms: race1, race2, race3, race4, race5, race6
b1=pd.DataFrame({"race1": [0 for i in range(n)]})
b2=pd.DataFrame({"race2": [0 for i in range(n)]})
b3=pd.DataFrame({"race3": [0 for i in range(n)]})
b4=pd.DataFrame({"race4": [0 for i in range(n)]})
b5=pd.DataFrame({"race5": [0 for i in range(n)]})
b6=pd.DataFrame({"race6": [0 for i in range(n)]})
#On va parcourir les éléments de notre dataset. de taille n. 
for k in range(n):
    #Pour chaque élément: On va regarder leurs valeurs dans le colonnes de races
    #On a 5 colonnes de race. 
    valrace1=dataset.loc[k, "applicant_race_1"]
    valrace2=dataset.loc[k, "applicant_race_2"]
    valrace3=dataset.loc[k, "applicant_race_3"]
    valrace4=dataset.loc[k, "applicant_race_4"]
    valrace5=dataset.loc[k, "applicant_race_5"]
    #et le stocker dans une liste
    L=[valrace1,valrace2,valrace3,valrace4,valrace5]
    #On va alors parcourir cette liste et en fonction de ce qu'elle contient on va remplir 
    #ou non les 6 colonnes crées avant
    #avec 1 si oui l'élément a cette race et on fait rien sinon sinon. 
    #pour race1: 
    if 1 in L:
         b1.loc[k,"race1"]=1
    #pour race2: 
    if 2 in L:
         b2.loc[k,"race2"]=1
    #pour race1: 
    if 3 in L:
         b3.loc[k,"race3"]=1
    #pour race1: 
    if 4 in L:
         b4.loc[k,"race4"]=1
    #pour race1: 
    if 5 in L:
         b5.loc[k,"race5"]=1
    #pour race1: 
    if 6 in L:
         b6.loc[k,"race6"]=1

#ensuite on enlève toutes nos colonnes de races de notre dataset
#Soit les colonnes: 
dataset=dataset.drop(columns=["applicant_race_1","applicant_race_2","applicant_race_3","applicant_race_4","applicant_race_5"])
#et on lui ajoute toutes nos nouvelles colonnes. 
dataset=dataset+b1+b2+b3+b4+b5+b6"""

#On fait pareil pour la race du co applicant. Mais cette fois ci avec 8 catégories. 
#On ne va pas faire comme dans le dataset. On va créer 8 nouvelles features.
#Car on a quand même plus de 150 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
#Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 
#Et sinon bien sur on met un 1 dans notre provided. 
#On met des co et des c partout comparé à au dessus. 
n=len(dataset)
print(n)
#On va créer nos 6 nouvelles colonnes:
#de longueur n et de noms: race1, race2, race3, race4, race5, race6
c1=pd.DataFrame({"corace1": [0 for i in range(n)]})
c2=pd.DataFrame({"corace2": [0 for i in range(n)]})
c3=pd.DataFrame({"corace3": [0 for i in range(n)]})
c4=pd.DataFrame({"corace4": [0 for i in range(n)]})
c5=pd.DataFrame({"corace5": [0 for i in range(n)]})
c6=pd.DataFrame({"corace6": [0 for i in range(n)]})
#On va parcourir les éléments de notre dataset. de taille n. 
for k in range(n):
    #Pour chaque élément: On va regarder leurs valeurs dans le colonnes de races
    #On a 5 colonnes de race. 
    covalrace1=dataset.loc[k, "co_applicant_race_1"]
    covalrace2=dataset.loc[k, "co_applicant_race_2"]
    covalrace3=dataset.loc[k, "co_applicant_race_3"]
    covalrace4=dataset.loc[k, "co_applicant_race_4"]
    covalrace5=dataset.loc[k, "co_applicant_race_5"]
    #et le stocker dans une liste
    Lco=[covalrace1,covalrace2,covalrace3,covalrace4,covalrace5]
    #On va alors parcourir cette liste et en fonction de ce qu'elle contient on va remplir 
    #ou non les 6 colonnes crées avant
    #avec 1 si oui l'élément a cette race et on fait rien sinon sinon. 
    #pour race1: 
    if 1 in Lco:
         c1.loc[k,"corace1"]=1
    #pour race2: 
    if 2 in Lco:
         c2.loc[k,"corace2"]=1
    #pour race1: 
    if 3 in Lco:
         c3.loc[k,"corace3"]=1
    #pour race1: 
    if 4 in Lco:
         c4.loc[k,"corace4"]=1
    #pour race1: 
    if 5 in Lco:
         c5.loc[k,"corace5"]=1
    #pour race1: 
    if 6 in Lco:
         c6.loc[k,"corace6"]=1

#ensuite on enlève toutes nos colonnes de races de notre dataset
#Soit les colonnes: 
dataset=dataset.drop(columns=["co_applicant_race_1","co_applicant_race_2","co_applicant_race_3","co_applicant_race_4","co_applicant_race_5"])
#et on lui ajoute toutes nos nouvelles colonnes. 
dataset=dataset+c1+c2+c3+c4+c5+c6


#On enlève les colonnes:
#denial_reason_1, denial_reason_2, denial_reason_3 et rate_spread
#car beaucoup trop de données manquantes
dataset = dataset.drop(columns=["denial_reason_1", "denial_reason_2", "denial_reason_3", "rate_spread"])

#que nous n'ayons plus que des données numerical
#pas de problèmes. 

28632


C:\Users\USER\AppData\Local\Temp\ipykernel_11536\1692075978.py:50: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dataset[["msamd", "county_code", "census_tract_number"]] = dataset[["msamd", "county_code", "census_tract_number"]].applymap(f)


28632
28632


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   action_taken                    0 non-null      float64
 1   agency_code                     0 non-null      float64
 2   applicant_ethnicity             0 non-null      float64
 3   applicant_income_000s           0 non-null      float64
 4   applicant_sex                   0 non-null      float64
 5   application_date_indicator      0 non-null      float64
 6   census_tract_number             0 non-null      object 
 7   co_applicant_ethnicity          0 non-null      float64
 8   co_applicant_sex                0 non-null      float64
 9   corace1                         0 non-null      float64
 10  corace2                         0 non-null      float64
 11  corace3                         0 non-null      float64
 12  corace4                         

In [11]:
#d. Création de la pipeline
#on créer et utiliser un modèle très simple ici le but surtout était de faire du feature engineering. 
#Une simple régression linéaire peut être une bonne idée. 
#On rappel qu'ici il s'agit de créer une ia supervisée. A voir les modèles auquels cela correspond. 

In [12]:
#e. Création des datasets d’entraînement et de test


In [13]:
#f. Séparation des target des datas dans les datasets


In [14]:
#g. Entraînement


In [15]:
#h. Evaluation sur les données tests


In [16]:
# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('rfc', RandomForestClassifier())
])

# data preparation
target_name = "action_taken"
data, target = dataset.drop(columns=target_name), dataset[target_name]


# training
model_pipeline_full.fit(data,target)

# predictive model
y_pred=model_pipeline_full.predict(data)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    data,
    target,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

ValueError: Input y contains NaN.